In [4]:
import pickle

In [18]:
import pandas as pd

In [1]:
csv_name='train_v21'
label_col='SalePrice'

In [2]:
from keras.models import load_model
model = load_model('lime_analysis_data/train_v21.h5')

Using TensorFlow backend.


In [5]:
with open('lime_analysis_data/complete_data.pkl', 'rb') as handle:
    complete_data = pickle.load(handle)

In [6]:
x_train = complete_data['xtrain']
x_test = complete_data['xtest']
x_val = complete_data['xval']
y_train = complete_data['ytrain']
y_test = complete_data['ytest']
y_val = complete_data['yval']
# param_dict = complete_data['params_dict']

# LIME

In [7]:
selected_cols = x_val.columns

In [8]:
x_val_copy = x_val.copy()

In [9]:
x_val_copy.reset_index(drop=True, inplace=True)

In [10]:
import lime
import lime.lime_tabular
import numpy as np
#explainer = lime.lime_tabular.LimeTabularExplainer(np.array(x_train), feature_names=selected_cols[:383])
explainer = lime.lime_tabular.LimeTabularExplainer(np.array(x_train), feature_names=selected_cols, class_names=[], verbose=True, mode='regression')
# i = np.random.randint(0, np.array(x_valid).shape[0])
# exp = explainer.explain_instance(x_valid.loc[10,:], predict, num_features=383)
# exp = explainer.explain_instance(x_valid.loc[0].values, model.predict, num_features=100)

In [19]:
def featur_im(explainer,x_valid):
    right_vl = []
    predict_local = []
    for im in range(x_valid.shape[0]):
        print('---------------',im,'---------------')
        exp = explainer.explain_instance(x_valid.loc[im], predict, num_features=x_valid.shape[1])
        print('='*10,len(exp.as_list()),'='*10)
        name_pos = list(x_valid.columns)
        right_vl.append(float(exp.predicted_value)) 
        predict_local.append(float(exp.local_pred[0]))
        intansity = [0]*len(name_pos)
        grt = [0]*len(name_pos)
        grt_and_eql = [0]*len(name_pos)
        less = [0]*len(name_pos)
        less_and_eql = [0]*len(name_pos)
        try:
            for i in exp.as_list():
                if i[0].find(' < ') != -1 and i[0].find(' <= ') != -1:
                    grt[name_pos.index(i[0][i[0].find(' < ')+3:i[0].find(' <= ')])] = float(i[0][0:i[0].find(' < ')])
                    less_and_eql[name_pos.index(i[0][i[0].find(' < ')+3:i[0].find(' <= ')])] = float(i[0][i[0].find(' <= ')+4:])
                    intansity[name_pos.index(i[0][i[0].find(' < ')+3:i[0].find(' <= ')])] = i[1]
                elif i[0].find(' <= ') != -1 and i[0].find(' < ') != -1:
                    grt_and_eql[name_pos.index(i[0][i[0].find(' <= ')+4:i[0].find(' < ')])] = float(i[0][0:i[0].find(' <= ')])
                    less[name_pos.index(i[0][i[0].find(' <= ')+4:i[0].find(' < ')])] = float(i[0][i[0].find(' < ')+3:])
                    intansity[name_pos.index(i[0][i[0].find(' <= ')+4:i[0].find(' < ')])] = i[1]
                elif i[0].find(' < ') != -1:
                    less[name_pos.index(i[0][0:i[0].find(' < ')])] = float(i[0][i[0].find(' < ')+3:])
                    intansity[name_pos.index(i[0][0:i[0].find(' < ')])] = i[1]
                elif i[0].find(' <= ') != -1:
                    less_and_eql[name_pos.index(i[0][0:i[0].find(' <= ')])] = float(i[0][i[0].find(' <= ')+4:])
                    intansity[name_pos.index(i[0][0:i[0].find(' <= ')])] = i[1]
                elif i[0].find(' > ') != -1:
                    grt[name_pos.index(i[0][0:i[0].find(' > ')])] = float(i[0][i[0].find(' > ')+3:])
                    intansity[name_pos.index(i[0][0:i[0].find(' > ')])] = i[1]
                elif i[0].find(' >= ') != -1:
                    grt_and_eql[name_pos.index(i[0][0:i[0].find(' >= ')])] = float(i[0][i[0].find(' >= ')+4:])
                    intansity[name_pos.index(i[0][0:i[0].find(' >= ')])] = i[1]
        except:
            pass
        if im == 0:
            intensity_dic = {'feature_name':name_pos, 'intensity0':intansity}
            df_int = pd.DataFrame(intensity_dic)
            val_dic = { '<f0':grt,'<=f0':grt_and_eql,"0f<":less,"0f<=":less_and_eql,'feature_name':name_pos}
            df_val = pd.DataFrame(val_dic)
            pr_dict = {'Right'}
        else:
            df_int['intensity'+str(im)] = intansity
            df_val['<f'+str(im)] = grt
            df_val['<=f'+str(im)] = grt_and_eql
            df_val[str(im)+"f<"] = less
            df_val[str(im)+"f<="] =less_and_eql
    local_and_Rpredict = {'prediction local':predict_local,'right':right_vl}
    df_lcl_right = pd.DataFrame(local_and_Rpredict)
    return df_int.T,df_lcl_right.T,df_val.T

In [15]:
def predict(qc):
    global model
    qc = model.predict(qc)
    return qc.reshape(qc.shape[0])

In [20]:
df_int1,df_lcl_right,df_val = featur_im(explainer,x_valid=x_val_copy)

--------------- 0 ---------------
Intercept 318429.7132539833
Prediction_local [228446.31351003]
Right: 229647.48
========== 396 ==========
--------------- 1 ---------------
Intercept 254087.22527332616
Prediction_local [212342.99329588]
Right: 199378.86
========== 396 ==========
--------------- 2 ---------------
Intercept 24917.355971349316
Prediction_local [225248.99571182]
Right: 219710.56
========== 396 ==========
--------------- 3 ---------------
Intercept 6897.98316702398
Prediction_local [187397.10377162]
Right: 193297.86
========== 396 ==========
--------------- 4 ---------------
Intercept 256919.48426115286
Prediction_local [154749.17581916]
Right: 156162.72
========== 396 ==========
--------------- 5 ---------------
Intercept 233694.12905152657
Prediction_local [146262.59536867]
Right: 114518.49
========== 396 ==========
--------------- 6 ---------------
Intercept 177949.5055448405
Prediction_local [209793.37344501]
Right: 225652.39
========== 396 ==========
--------------- 7

Intercept 288912.18312635
Prediction_local [210153.85948001]
Right: 206015.38
========== 396 ==========
--------------- 59 ---------------
Intercept -16422.86433497077
Prediction_local [290970.84161203]
Right: 325719.72
========== 396 ==========
--------------- 60 ---------------
Intercept 236636.52207282826
Prediction_local [180629.2569547]
Right: 184510.02
========== 396 ==========
--------------- 61 ---------------
Intercept 119200.52080998899
Prediction_local [168013.5164055]
Right: 186495.8
========== 396 ==========
--------------- 62 ---------------
Intercept 103752.05015170385
Prediction_local [289756.56197329]
Right: 322364.34
========== 396 ==========
--------------- 63 ---------------
Intercept 239042.31046382088
Prediction_local [156042.72403669]
Right: 164713.17
========== 396 ==========
--------------- 64 ---------------
Intercept 176961.6358468511
Prediction_local [153432.24037153]
Right: 150018.83
========== 396 ==========
--------------- 65 ---------------
Intercept 338

Intercept 146026.30701871365
Prediction_local [206327.10935098]
Right: 164305.62
========== 396 ==========
--------------- 117 ---------------
Intercept 136084.57196499975
Prediction_local [206594.74432679]
Right: 178526.81
========== 396 ==========
--------------- 118 ---------------
Intercept 104478.40296768106
Prediction_local [112437.12568132]
Right: 113627.92
========== 396 ==========
--------------- 119 ---------------
Intercept 184989.8701280903
Prediction_local [236439.62847293]
Right: 292850.25
========== 396 ==========
--------------- 120 ---------------
Intercept 91947.6091394217
Prediction_local [118760.37459301]
Right: 117389.46
========== 396 ==========
--------------- 121 ---------------
Intercept 215803.97219174216
Prediction_local [140405.85669042]
Right: 134123.42
========== 396 ==========
--------------- 122 ---------------
Intercept 96815.17403593015
Prediction_local [162649.75028524]
Right: 153287.69
========== 396 ==========
--------------- 123 ---------------
Int

Intercept 136521.65960522194
Prediction_local [291500.27278135]
Right: 290225.53
========== 396 ==========
--------------- 175 ---------------
Intercept 89482.72681288655
Prediction_local [259052.32591627]
Right: 241372.38
========== 396 ==========
--------------- 176 ---------------
Intercept 88417.94483144165
Prediction_local [154555.05956871]
Right: 139344.12
========== 396 ==========
--------------- 177 ---------------
Intercept 271646.9654449462
Prediction_local [185395.82078097]
Right: 186784.42
========== 396 ==========
--------------- 178 ---------------
Intercept 163784.83229395776
Prediction_local [142320.67468068]
Right: 128586.89
========== 396 ==========
--------------- 179 ---------------
Intercept 107564.86191661382
Prediction_local [185522.23020949]
Right: 211202.62
========== 396 ==========
--------------- 180 ---------------
Intercept 107601.53700453429
Prediction_local [190385.354215]
Right: 181473.69
========== 396 ==========
--------------- 181 ---------------
Inte

Intercept 353623.08127064304
Prediction_local [188309.25585299]
Right: 168450.11
========== 396 ==========
--------------- 233 ---------------
Intercept 288231.95440000045
Prediction_local [119868.56581243]
Right: 117937.12
========== 396 ==========
--------------- 234 ---------------
Intercept 20733.463141578948
Prediction_local [210679.48926584]
Right: 214484.31
========== 396 ==========
--------------- 235 ---------------
Intercept 412491.1610496125
Prediction_local [234591.90664318]
Right: 225784.48
========== 396 ==========
--------------- 236 ---------------
Intercept 234359.69451645075
Prediction_local [200115.08859207]
Right: 190333.36
========== 396 ==========
--------------- 237 ---------------
Intercept -18504.312188724783
Prediction_local [229857.02835677]
Right: 225067.69
========== 396 ==========
--------------- 238 ---------------
Intercept 132528.1685933309
Prediction_local [282191.65602413]
Right: 383853.84
========== 396 ==========
--------------- 239 ---------------


# Saving results

In [21]:
results = {}
results['intensity_df'] = df_int1
results['right_local_pred_df'] = df_lcl_right
results['feature_value_df'] = df_val

In [41]:
with open("lime_analysis_data/results.pkl", 'wb') as handle1:
    pickle.dump(results, handle1, protocol=pickle.HIGHEST_PROTOCOL)

# Losses of Best Data on Validation and Test data

In [42]:
loss_valid, mse_valid = best_model.evaluate(x_valid,y_valid)

rmse_valid = loss_valid ** 0.5

rmse_valid

NameError: name 'best_model' is not defined

In [152]:
loss_test, mse_test = best_model.evaluate(x_test,y_test)

rmse_test = loss_test ** 0.5

rmse_test

291/291 [==============================] - 0s 86us/step


30371.172441377985